# Декораторы

В данной лекции речь пойдет о декораторах, о цели их использования и способах создания. Однако, прежде, чем начать этот разговор, нам необходимо вспомнить материал [прошлой лекции](../lesson7/interactive_conspect.ipynb), посвященный облостям видимости вложенным функциям.

## Области видимости: повторение

Начнем детальное знакомство с замыканиям с небольшого уточнения информации об областях видимости. На прошлой лекции мы разобрались, что у любой функции есть свое пространство имен, которое можно представить, как перечень переменных, используемых в теле функции - локальных переменных. Пространство имен создается при каждом вызове функции и уничтожается в момент завершения выполнения функции. Переменные, которые определены на уровне модуля, являются глобальными переменнами по отношению к данной функции. При этом в теле функции можно получить доступ к глобальным переменным:

In [1]:
var_global = 5

def print_vars(var_local: int) -> None:
    print(f'{var_local = };')
    print(f'{var_global = };')

In [2]:
print_vars(10)

var_local = 10;
var_global = 5;


Если глобальная переменная является объектом изменяемого типа данных, действия в теле функции могут привести к изменению значения данной глобальной переменной. Однако стоит понимать, что нового объекта создано не будет. Вы не можете осуществить перепревязку глобальной переменной в теле функции. Подобные попытки приведут лишь к созданию локальной переменной, с тем же идентефикатором, что и глобальная переменная, в теле функции. Локальная переменная будет связана с новым объектом в памяти, время жизни которого ограничено телом функции. Данная локальная переменная в теле функции будет перекрывать своим именем имя глобальной переменной. При этом глобальная переменная никак не изменится. Эти положения иллюстрируются следующим примером:

In [4]:
list_global = [1, 2]


def print_list_info(list_value: list, list_name: str) -> None:
    print(
        f'{list_name} value: {list_value};',
        f'{list_name} id: {id(list_value)};',
        sep='\n',
        end='\n\n',
    )


def change_list() -> None:
    list_global = list(range(10))
    print_list_info(list_global, 'list_global')

In [5]:
print_list_info(list_global, 'list_global')
change_list()
print_list_info(list_global, 'list_global')

list_global value: [1, 2];
list_global id: 140667361610688;

list_global value: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9];
list_global id: 140667119931264;

list_global value: [1, 2];
list_global id: 140667361610688;



Более того, Python не определяет динамически, является ли переменная локальной или нет. Данная информация определяется в момент определения функции и остается неизменной на протяжении всей жизни программы. Отсюда следует, что любая переменная в теле функции является или глобальной, или локальной, она не может быть локальной в одних случая, а глобальной - в других, или быть до определенного момента глобальной, а потом локальной. Переменная считается локальной, если является операндом оператора присваивания в любой форме (как простой, так и составной). Данное положение можно проиллюстрировать следующим примером:

In [13]:
some_number = 9


def func1(num: int) -> None:
    print(f'{num = };')
    print(f'{some_number = };')
    print('')


def func2(num: int) -> None:
    some_number = 6
    print(f'{num = };')
    print(f'{some_number = };')
    print('')


def func3(num: int) -> None:
    print(f'{num = };')
    print(f'{some_number = };')
    some_number = 6
    print('')

In [14]:
func1(3)
func2(3)
func3(3)

num = 3;
some_number = 9;

num = 3;
some_number = 6;

num = 3;


UnboundLocalError: local variable 'some_number' referenced before assignment

Как мы видим любая попытка перепривязки или составного присваивания в теле функции делает переменную локальной. Чтобы избежать подобного поведения и получить возможность изменять глобальные переменные в теле функции, существует специальное слово **global**, явно сообщающее интерпретатору, что переменная является глобальной и поиска данного имени должен осуществляться на уровне модуля. Исправим функцию из примера выше:

In [16]:
def func3(num: int) -> None:
    global some_number

    print(f'{num = };')
    print(f'{some_number = };')
    some_number = 6
    print('')

In [17]:
func3(3)

print(f'{some_number = };')

num = 3;
some_number = 9;

some_number = 6;


Аналогичная ситуация складывается и с вложенными функциями - функциями, определенными в телах других функций. Вложенная функция может иметь доступ к переменным, находящимся в области видимости внешней функции, т.е. функции, в которую данная функция вложена. Однако любые попытки перепривязки приводят к тому, что внешние переменная воспринимаются интерпретатором как локальные. Данное положение можно проиллюстрировать следующим примером:

In [18]:
from typing import Callable


def outer_func1(num: int) -> Callable:
    outer_num = num

    def inner_func(num: int) -> None:
        print(f'inner {num = };')
        print(f'{outer_num = };')
        print('')

    print(f'{outer_num = };', end='\n\n')

    return inner_func


def outer_func2(num: int) -> Callable:
    outer_num = num

    def inner_func(num: int) -> None:
        print(f'inner {num = };')
        print(f'{outer_num = };')
        outer_num = 5
        print('')

    print(f'{outer_num = };', end='\n\n')

    return inner_func

In [19]:
inner_func1 = outer_func1(10)
inner_func2 = outer_func2(10)

inner_func1(3)
inner_func2(3)

outer_num = 10;

outer_num = 10;

inner num = 3;
outer_num = 10;

inner num = 3;


UnboundLocalError: local variable 'outer_num' referenced before assignment

Для решения данной проблемы, по аналогии с global, было введено специальное слово **nonlocal**, которое явно сообщает интепретатору, что используемое имя содержится в одной из внешних функций. В отличие от global, поиск нужного имени осуществляется последовательно во всех функциях, в которые была вложена данная функция, но не на уровне модуля. Имя это ввиду исправим функции из предыдущего примера:

In [25]:
def outer_func2(num: int) -> Callable:
    outer_num = num

    def inner_func(num: int) -> None:
        nonlocal outer_num

        print(f'inner {num = };')
        print(f'{outer_num = };')
        outer_num = 5
        print('')

    print(f'{outer_num = };', end='\n\n')

    return inner_func

In [26]:
inner_func2 = outer_func2(10)
inner_func2(3)

outer_num = 10;

inner num = 3;
outer_num = 10;



## Замыкания

Все вложенные функции из примеров выше, использовавшие нелокальные переменная, являются замыканиями. Формально замыкание можно определить следующим образом: замыкание - это функция с расширенной областью видимости, которая охватывает все неглобальные переменные, на которые есть ссылки в теле функции, хотя они в нем не определены. Т.е. замыкания - это такие функции, которые могут обращаться к неглобальными переменным, определенным вне их тела. 

Давайте рассмотрим более содержательный пример использования замыкания. Допустим мы хотим создать функцию счетчик, которая будет хранить в себе актуальную информацию о количестве вызовов, обновлять и возвращать ее при каждом новом вызове. Сама по себе эта функция несет мало пользы, поэтому допустим что она является частью большой и важной системы, а вызывающая сторона использует эту информацию для учета каких-то статистик. Как бы могла выглядеть эта функция-счетчик?

Мы не можем поместить переменную, хранящую информацию о количестве вызовов в тело нашей функции-счетчика, поскольку пространство имен каждый раз создается заново, а значит не сохраняет никакой информации о предыдущих вызовах. Первое, что приходит в голову - сделать переменную-счетчик глобальной:

In [27]:
counter = 0


def count() -> int:
    global counter
    
    counter += 1
    return counter

In [34]:
count()

7

Отлично! Это работает! Но что если мы захотим создать два независимых счетчика, которые бы могли подсчитывать вызовы и хранить информацию независимо друг от друга?

In [35]:
counter1 = count
counter2 = count

In [36]:
print(counter1())
print(counter2())

8
9


В этом случае мы помним предыдущие состояния функции. Более того, каждый вызов "независимого счетчика" влияет на состояние остальных счетчиков. Т.е. использование глобальных переменных нашу проблему не решает. В такие моменты замыкания и становятся достаточно полезными интсрументами:

In [37]:
from typing import Callable


def make_counter() -> Callable[[], int]:
    counter = 0

    def count() -> int:
        nonlocal counter
        counter += 1

        return counter
    
    return count

In [38]:
counter1 = make_counter()
counter2 = make_counter()

for i in range(3):
    counter1()

for i in range(5):
    counter2()

print(f'counter1: {counter1()};')
print(f'counter1: {counter2()};')

counter1: 4;
counter1: 6;


Теперь создание функции счетчика происходит в момент выполнения программы посредством вызова функции `make_counter()`, в теле которой объявляется переменная `counter` и функция `count()`. Функция `count()` является замыканием, поскольку она вложена в функцию make_counter() и использует в своем теле внешние неглобальные переменные - counter, - для хранения и обновления информации о количестве вызовов. Функции подобные `make_counter()` называются "фабриками", посколько позволяют "производить" некоторые объекты по ходу выполнения программы - в нашем случае, функции-счетчики. 

Данный подход полностью решает поставленную проблему: мы можем создавать функции-счетчики, которые позволяют получать информацию о текущем количестве вызовов, при необходимости мы можем создать сколько угодно независимых счетчиков. Однако данный подход также должен вызвать у вас ряд вопросов в числе которых: а где же хранится информация о количестве вызовов? Ведь переменная `counter` определена в теле функции make_counter, а значит является локальной переменной, т.к. должна быть уничтожена после выполнения функции. Но почему-то этого не происходит, и мы по-прежнему имеем к ней доступ через замыкание.

Это происходит потому что внутри функции `count` переменная `counter` является свободной переменной, т.е. несвязанной локальной областью видимости. Мы можем наглядно в этом убедиться, произведя инспекцию нашей функции:

In [39]:
print(f'local variables: {counter1.__code__.co_varnames}')
print(f'free variables: {counter1.__code__.co_freevars}')

local variables: ()
free variables: ('counter',)


Python хранит откомпилировнаное тело функции в атрибуте `__code__`. Также в этом атрибуте хранится информация о локальных и свободных переменных. Как мы видим в примере выше переменная `counter` действительно является свободной переменной. Однако таким образом python сохраняет именно имена переменных, но где же хранится само значение?

Привязка переменной counter сохраняется в специальном атрибуте `__closure__`(буквально - замыкание). Между элементами `__closure__` и именами, хранящимися в `__code__.co_freevars` существует взаимооднозначное соответствие. Элементами атрибута `__closure__` являются специальные ячейки (cells). У каждой ячейки есть атрибут `cell_content` - содержимое ячейки, именно в этом атрибуте и хранится значение свободной переменной. 

In [41]:
print(f'counter1 closure: {counter1.__closure__}')

for i, cell in enumerate(counter1.__closure__):
    print(f'counter1 cell{i} content: {cell.cell_contents};')

counter1 closure: (<cell at 0x7fefac20e370: int object at 0x7fefb2030990>,)
counter1 cell0 content: 4;


Таким образом замыкания хранят значения свободных переменных в специальном кортеже и специальным образом. Именно поэтому мы можем продолжать использовать переменные внешних функций за пределами тела этих внешних по отношению к замыканию функций.

Резюмируем: замыкание – это функция, которая запоминает привязки свободных переменных, существовавших на момент определения функции, так что их можно использовать впоследствии при вызове функции, когда область видимости, в которой она была определена, уже не существует.

## Декораторы

Разобравшись с замыканием мы готовы к изучению декораторов. Декораторы - это вызываемый объект, который принимает другую функцию - декорируемую функцию, в качестве аргумента. Обычно декораторы используются, чтобы дополнить фнкционал уже существующих функций некоторым образом, в этом случае в результате применения декоратора исходная функция заменяется на результат вычисления декоратора. Также декораторы могу использоваться, чтобы осуществить некоторые дополнительные манипуляции с функцией по определенному общему сценарию.

Рассмотрим следующий учебный пример:

In [42]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print('start function')
        result = func(*args, **kwargs)
        print('finished function')

        return result
    
    return wrapper


@my_decorator
def do_something():
    print('do_something')

In [43]:
do_something()

start function
do_something
finished function


В данном примере мы реализовали просто декоратор - my_decorator. Данный декоратор принимает на вход функцию. В своем теле он определяет замыкание, которое использует данную функцию и выводит в стандартный поток вывода сообщения о начале вычислений и об их завершении. Также в замыкании происходит вызов декорируемой функции и возвращение результатов ее вычисления, что является распространненой практикой, поскольку декораторы именно дополняеют поведение функции, а не переопределяют его. Сам декоратор возвращает замыкание в качестве результата вычисления.

Вызов декоратора, позволяющий применить его к нашей функции, является всего лишь синтаксической глазурью:

```python
@my_decorator
def do_something():
    ...
```

Формально, ничего не мешает нам применить декоратор следующим образом:

In [44]:
def do_something():
    print('do_something')

do_something = my_decorator(do_something)

In [45]:
do_something()

start function
do_something
finished function


Поскольку код применения декораторов из примеров выше эквивалентен, можно сделать следующий вывод: объект, возвращаемый декоратором заменяет собой декорируемую функцию. Это можно проверить, обратившись к имени функции после декорировнаия:

In [54]:
do_something.__name__

'wrapper'

Т.е. функция `wrapper`, определенная в теле декоратора, действительно заменила собой исходный объект. Именно поэтому мы и определили замыкание с настолько общей сигнатурой. Ведь функция wrapper будет заменять собой любую декорируемую функцию, а значит должна обеспечить корректную работу функций с любыми сигнатурами. 

Из факта подобной замены следует, что затирается и сигнатура первоначально функции, а значит IDE не сможет предоставлять нам интеллектуальные подсказки о декорируемой функции, типах ее аргументов и возвращаемом значении. Чтобы это предотвратить, в стандартной библиотеки был реализован декоратор `wraps`, который позволяет сохранить информацию о декорированной функции.

Чтобы посмотреть на использования описанного декоратора, давайте реализуем декоратор, распечатывающий информации о вызове функции: имя функции, с какими аргументами был совершен вызов, за какое время была выполнена фукнция.

In [46]:
import time

from typing import Callable
from functools import wraps


def call_info(func) -> Callable:
    @wraps(func)
    def get_call_info(*args, **kwargs) -> Callable:
        args_list = []

        if args:
            args_list.append(', '.join(str(arg) for arg in args))

        if kwargs:
            args_list.append(
                ', '.join(f'{key}={val}' for key, val in kwargs.items())
            )

        args_str = ', '.join(args_list)

        time_start = time.time()
        result = func(*args, **kwargs)

        print(
            f'[CALL INFO]: {func.__name__}({args_str}) -> {result} '
            f'|| {time.time() - time_start}'
        )

        return result

    return get_call_info

In [47]:
from time import sleep


@call_info
def do_something() -> None:
    sleep(1)

In [50]:
do_something()

[CALL INFO]: do_something() -> None || 1.0010740756988525


In [51]:
do_something.__name__

'do_something'

## Момент выполнения декораторов

Декораторы выполняются в момент определения функций. Обычно, когда программа состоит из нескольких модулей (разбита на несколько файлов с расширением .py), вы импортируете декорированные функции из других модулей. В данной ситуации декоратор выполняется в момент импорта. Важно осознавать, что декоратор будет выполняться отдельно для каждой декорируемой функции. В качестве демонстрации этого принципа приведен следующий пример:

In [52]:
def decorate(func):
    print('run decorate')
    return func


@decorate
def do_something() -> None:
    print('do_something')


@decorate
def do_another_thing() -> None:
    print('do_another_thing')

run decorate
run decorate


In [53]:
do_something()

do_something


In [54]:
do_another_thing()

do_another_thing


Как вы видите сообщение о запуске декоратора было распечатано дважды в момент определения функции. Сами же функции выполняются только посредством явного вызова в коде. Этот пример интересен тем, что возвращаемым объектом декоратора служит исходная функция, а не замыкание, определяемое в теле. В контексте пример выше это кажется упрощением, но подобная техника может оказаться очень полезной, например для регистрации функций в общем реестре функций. Более практически полезный пример будет разберан на семинаре.

## Композиции декораторов

При необходимости вы можете применять несколько декораторов к функции одновременно. 

In [59]:
def outer(func):
    def wrapper(*args, **kwargs):
        print('outer')
        
        result = func(*args, **kwargs)
        return result
    
    return wrapper


def inner(func):
    def wrapper(*args, **kwargs):
        print('inner')

        result = func(*args, **kwargs)
        return result
    
    return wrapper


@inner
@outer
def do_something() -> None:
    print('do_something')

In [60]:
do_something()

inner
outer
do_something


Порядок применения декораторов в данном примере хорошо описывается следующим эквивалентным кодом:

In [57]:
def do_something() -> None:
    print('do_something')

do_something = outer(inner(do_something))

In [58]:
do_something()

outer
inner
do_something


## Параметризованные декораторы

В некоторых ситуациях бывает полезно иметь возможность настраивать декоратор и передавать в него какие-либо значения помимо декорируемой функции. На этот случай все, что нам нужно сделать - реализовать параметризованный декоратор. Формально, параметризованный декоратор - не декоратор, а фабрика декораторов. Определение данного объекта выглядит следующим образом:

In [62]:
import time

from typing import Callable
from functools import wraps


def call_info(
    use_arguments: bool = True,
    fix_time: bool = True,
) -> Callable:
    def call_deco(func) -> Callable:
        @wraps(func)
        def get_call_info(*args, **kwargs) -> Callable:
            args_list = []

            if args and use_arguments:
                args_list.append(', '.join(str(arg) for arg in args))

            if kwargs and use_arguments:
                args_list.append(
                    ', '.join(f'{key}={val}' for key, val in kwargs.items())
                )

            args_str = ', '.join(args_list)

            time_start = time.time()
            result = func(*args, **kwargs)

            dots = '...'

            print(
                f'[CALL INFO]: {func.__name__}'
                f'({args_str if use_arguments else dots}) -> {result}'
                f' || {time.time() - time_start if fix_time else dots}'
            )

            return result

        return get_call_info
    
    return call_deco

In [63]:
from random import randint
from time import sleep


@call_info()
def sum_numbers(*args) -> float:
    return sum(args)


@call_info(fix_time=False)
def invert_numbers(*args) -> list[float]:
    return [
        1 / i if i != 0 else i for i in args
    ]


@call_info(use_arguments=False)
def do_something() -> None:
    time_to_sleep = randint(1, 5)
    sleep(time_to_sleep)

    print('did something')

In [64]:
sum_numbers(1, 2, 3, 4, 5)
invert_numbers(1, 2, 4, 0)
do_something()

[CALL INFO]: sum_numbers(1, 2, 3, 4, 5) -> 15 || 6.4373016357421875e-06
[CALL INFO]: invert_numbers(1, 2, 4, 0) -> [1.0, 0.5, 0.25, 0] || ...
did something
[CALL INFO]: do_something(...) -> None || 3.015892267227173


В данных примерах использование декоратора выглядит как функциональный вызов. Это происходит потому что call_info не является декоратором сама по себе. Как было сказано ранее call_info - это фабрика декораторов, т.е. это функция, позволяющая создать декораторы в процессе выполнения программы. Созданные декоратор применяется к декорируемой функции.

Менее употребимый, но эквивалентный код, помогающий улучшить понимание работы параметрического декоратора приведен ниже:

In [65]:
from random import randint
from time import sleep

def do_something() -> None:
    time_to_sleep = randint(1, 5)
    sleep(time_to_sleep)

    print('did something')

do_something = call_info()(do_something)

In [66]:
do_something()

did something
[CALL INFO]: do_something() -> None || 3.002840280532837


И еще более подробно:

In [67]:
from random import randint
from time import sleep

def do_something() -> None:
    time_to_sleep = randint(1, 5)
    sleep(time_to_sleep)

    print('did something')

wrapper = call_info()
do_something = wrapper(do_something)

In [68]:
do_something()

did something
[CALL INFO]: do_something() -> None || 4.003225564956665
